## Filtering

In [1]:
import pandas as pd
import numpy as np
import os
import gc
import matplotlib.pyplot as plt
import time

from functions.filtering import *
from functions.plotting import *
from functions.fitting import *

_FOLDER = "data/"
_FOLDER_2 = "figures/"
_FOLDER_3 = "results/"
SAVE_FIGURES = False

R2_limit = 0.99

### Original data

In [2]:
drug_curves = pd.read_csv(_FOLDER+"normalised_dose_response_data.csv")

if "Unnamed: 0" in drug_curves:
    drug_curves.drop("Unnamed: 0", axis=1, inplace =True)

col_to_drop = ["per_slope_change_"+str(i) for i in range(8)]+\
            ["slope_" + str(i) for i in range(9)]    
drug_curves.drop(col_to_drop, axis=1, inplace=True)    
    
conc_columns= ["fd_num_"+str(i) for i in range(10)]
response_norm = ['norm_cells_'+str(i) for i in range(10)]
CCL_names = dict(zip(drug_curves["COSMIC_ID"], drug_curves["CELL_LINE_NAME"]))
df= pd.read_csv(_FOLDER+'Drug_Features.csv')
drug_names = dict(zip(df["Drug ID"].values, df["Drug Name"].values))
del df
drug_curves["drug_name"] = drug_curves["DRUG_ID"].map(drug_names)
drug_curves["CCL_name"] = drug_curves["COSMIC_ID"].map(CCL_names)
drug_curves.shape

(225384, 28)

## Filtering 1: 4 stage filtering

In [3]:
%%time
# difference between middle points 0.2
df_filt_1234 = filtering_sigmoid_curves(drug_curves, filtering_scenario=[1,2,3,4], \
                        response_columns = response_norm, \
                        first_points_lower_limit = 0.8, last_points_upper_limit = 0.2, 
                         middle_points_limit = -0.2)
df_filt_1234.to_csv(_FOLDER_3+"filt_1234_02.csv", index=False)

Original dataset: (225384, 28)
1st filtration (Ensure that all the response are less than 1): Filtered dataset: (63325, 28)
2d filtration (Ensure that first and last points form plateus): Filtered dataset: (6321, 30)
3d stage filtration (Specified location of the plateus): Filtered dataset: (2152, 30)
4th stage filtration (Cut off high ancedent points): Filtered dataset: (2108, 30)
CPU times: user 1.06 s, sys: 158 ms, total: 1.22 s
Wall time: 1.64 s


In [4]:
%%time
df = df_filt_1234.copy()
fitting_function = "sigmoid_4_param"

r2, fit_param = fitting_column(df, df.index, x_columns=conc_columns, y_columns= response_norm,
                               fitting_function = fitting_function, default_param=True)
df[fitting_function+"_r2"] = r2
df[fitting_function] = fit_param
df = df[df[fitting_function+"_r2"]>R2_limit]
print(df_filt_1234.shape, df.shape)
df.to_csv(_FOLDER_3+"fit_1234_02.csv", index=False)

100%|██████████| 2108/2108 [00:11<00:00, 183.81it/s]


<function sigmoid_4_param at 0x7fc355a15048>
(2108, 30) (1531, 32)
CPU times: user 9.69 s, sys: 546 ms, total: 10.2 s
Wall time: 12.1 s


## Filtering 2:  auc>0.7 and spearman_r<0

In [9]:
%%time
df_filt_auc = pd.read_csv(_FOLDER_3+"filt_auc_02.csv")
df = df_filt_auc.copy()
fitting_function = "sigmoid_4_param"

r2, fit_param = fitting_column(df, df.index, x_columns=conc_columns, y_columns= response_norm,
                               fitting_function = fitting_function, default_param=True)
df[fitting_function+"_r2"] = r2
df[fitting_function] = fit_param
df = df[df[fitting_function+"_r2"]>R2_limit]
print(df_filt_auc.shape, df.shape)
df.to_csv(_FOLDER_3+"fit_auc_02.csv", index=False)

100%|██████████| 12169/12169 [01:48<00:00, 111.82it/s]


<function sigmoid_4_param at 0x7fc355a15048>
(12169, 31) (3620, 33)
CPU times: user 1min 39s, sys: 7.14 s, total: 1min 46s
Wall time: 1min 51s


## Filtering 3: direct fitting

In [ ]:
functions = [
             "fsigmoid", 
             "sigmoid_2_param",
             "sigmoid_3_param",
             "sigmoid_4_param",
             "logistic_4_param",
            "ll4_4_param",
            "ll4R_4_param",
            "logLogist_3_param"]

In [ ]:
%%time
df_no_filt = compare_fitting_functions(drug_curves, functions, conc_columns, response_norm, 
                                       save_file_name = _FOLDER_3 +"fit_no_filt.csv")